# Forecast Consumption

Forecasting using timeseries model the consumption of water, gas and electricity.



## Preparing the data

In [ ]:
# Import the necessary libraries
import sys
import pandas as pd

sys.path.append('../../')
from core.settings import gas_monthly_df


In [14]:
# variables
from os import path


gas_monthly_df = pd.read_csv(r"/home/cyat/projects/CATY_Management/data_storage/resampled_data_D_gas.csv")


In [16]:
gas_monthly_df.info()
# gas_monthly_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4779 entries, 0 to 4778
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       4779 non-null   object 
 1   id_meter   4779 non-null   float64
 2   gas_m3     4779 non-null   float64
 3   calc_cons  4777 non-null   float64
dtypes: float64(3), object(1)
memory usage: 149.5+ KB


,id_meter,gas_m3,calc_cons
count,4779.000000,4779.000000,4777.000000
mean,2.786985,32518.952611,2.280152
std,3.052463,18751.101625,1.723407
min,1.000000,1.000000,0.000000
25%,1.000000,1600.591815,1.058448
50%,1.000000,41426.910455,1.388875
75%,8.000000,44938.382037,3.705097
max,8.000000,48156.325000,7.953000


In [26]:
# Dealing with missing values
# show NaN values rows
gas_monthly_df[gas_monthly_df.isnull().any(axis=1)]

# show rows for date 2014-09-04
gas_monthly_df[gas_monthly_df['date'] == '2014-09-04']

# # fill row 0 NaN value with 0
# gas_monthly_df.fillna(0, inplace=True)

# show duplicate dates rows
gas_monthly_df[gas_monthly_df.duplicated(['date'])]

# drop duplicate dates rows
gas_monthly_df.drop_duplicates(subset=['date'], inplace=False)


,date,id_meter,gas_m3,calc_cons
4750,2014-09-04,8.0,1593.752217,1.954171
4751,2014-09-05,8.0,1595.706387,1.954171
4752,2014-09-06,8.0,1597.660558,1.954171
4753,2014-09-07,8.0,1599.614729,1.954171
4754,2014-09-08,8.0,1601.568900,1.954171
4755,2014-09-09,8.0,1603.523071,1.954171
4756,2014-09-10,8.0,1605.477242,1.954171
4757,2014-09-11,8.0,1607.431413,1.954171
4758,2014-09-12,8.0,1609.385583,1.954171
4759,2014-09-13,8.0,1611.339754,1.954171


## Prepare the model